In [2]:
import matlab.engine
from nipype.interfaces.utility import Function
import nipype.pipeline.engine as pe
import nipype.interfaces.io as nio

In [50]:
#eng = matlab.engine.start_matlab()

def checkstr(string):
    if string[-1] == '/':
        return string
    else:
        return string + '/'

def setup(vbvs, gpml, depvb, comp, outpath):
    import os
    for i in [vbvs, gpml, depvb, comp]:
        eng.addpath(i)
    eng.run(checkstr(outpath) + 'startup.m', nargout=0)
    os.chdir(outpath)
    #return output
    
Setup = pe.Node(name='Setup',
                interface=Function(input_names=['vbvs','gpml',
                                                'depvb','comp','outpath'],
                                        output_names=[''],
                                        function=setup))

def csv(colnums, matfiles):
    import pandas as pd
    df = pd.DataFrame(columns=['colNum','matFn'])
    for i in range(colnums):
        df.loc[i] = [i, matfiles[i]]
    df.iloc[:,0] = df.iloc[:,0].astype(int)
    return df


Csv = pe.Node(name='Csv',
             interface=Function(input_names=['colnums','matfiles'],
                               output_names=['df'],
                               function=csv))

def runstep_bf(step, infile, outfile, data, colnum):
    eng.deployEndoPhenVB('step', step,
                        'inputMat', infile,
                        'colNum', colnum,
                        'outFile', outfile,
                        nargout = 0)

RunstepBF = pe.Node(name='Runstep',
                 interface=Function(input_names=['step',
                'infile','outfile','data','colnum'],
                output_names=[''],
                        function=runstep_bf))  

node_infosource = pe.Node(niu.IdentityInterface(fields=['colnum']), name = 'infosource')
node_infosource.iterables = [('colnum', [x for x in range(1,95)])]
node_iterfunc = pe.Node(niu.Function(input_names=['colnum'],
                                                output_names=[''],
                                                function=runstep_bf), name="iterfunc" )


    
if __name__ == '__main__':
    import argparse
    defstr = '(default %(default)s)'
    parser = argparse.ArgumentParser()
    parser.add_argument('-n', '--numrepeats', type=int, help='')
    parser.add_argument('-cn', '--colnums', type=int, help='the number of features in your data matrix')
    parser.add_argument('-s', '--step', type=str, help="either 'bf', 'normalize', or 'fxvb'")
    parser.add_argument('-v', '--vbvs', type=str, help='path to varbvs directory')
    parser.add_argument('-g', '--gpml', type=str, help='path to gpml directory')
    parser.add_argument('-d', '--depvb', type=str, help='path to where deployEndoPhenVB.m lives')
    parser.add_argument('-c', '--comp', type=str, help='path to where computeGPLnZHelper.m lives')
    parser.add_argument('-o', '--outpath', type=str, help='path for the output files')
    args=parser.parse_args()
    numrepeats = args.numrepeats
    colnums = args.colnums
    step = args.step
    vbvs = args.vbvs
    gpml = args.gpml
    depvb = args.depvb
    comp = args.comp
    outpath = args.outpath

Setup.inputs.vbvs = vbvs
Setup.inputs.gpml = gpml
Setup.inputs.depvb = depvb
Setup.inputs.comp = comp
Setup.inputs.outpath = outpath
Csv.inputs.colnums = colnums

wf = pe.Workflow(name='wf')
wf.base_dir = data_dir
wf.connect(node_infosource, 'colnum', node_iterfunc, 'colnum')
wf.run()
wf.write_graph()

usage: __main__.py [-h] [-n NUMREPEATS] [-cn COLNUMS] [-s STEP] [-v VBVS]
                   [-g GPML] [-d DEPVB] [-c COMP] [-o OUTPATH]
__main__.py: error: unrecognized arguments: -f /run/user/134974/jupyter/kernel-30d07614-d32a-40bf-a3fd-fb767a851579.json


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


In [6]:
def runstep(step, infile, outfile, data, colnum):
    

In [45]:
matfiles=['this/test','this/test1','this/test2']

csv(3, matfiles)
    

,colNum,matFn
0,0,this/test
1,1,this/test1
2,2,this/test2


In [16]:
import pandas as pd

In [22]:
df = pd.DataFrame(columns=['e','w'])

In [33]:
for i in range(3):
    df.loc[i] = [i,'we']

In [34]:
df

,e,w
0,0.0,we
1,1.0,we
2,2.0,we


In [1]:
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as niu
import nipype.interfaces.matlab as matlab

In [ ]:
'''
def test_wrap(step, inputmat, colnum, outfile):
    matlab = matlab.MatlabCommand()
    matlab.inputs.mfile=True
    mlab.inputs.script = \
                    """
                    arg_out = deployEndoPhenVB('step',%s,'inputMat',%s,'colNum',%s,'outFile');
                    """ % (step, inputmat, colnum, outfile) 
    res = matlab.run()
'''